# Developer
## Setup : this should seem really familiar

In [1]:
from dotenv import load_dotenv
from os import getenv

if load_dotenv():
    key, secret, url = getenv('qkey'), getenv('qsecret'), getenv('qurl')
    try:
        assert key, "Key not loaded"
        assert secret, "Secret not loaded"
        assert url, "Url not loaded"
        print("All environment variables loaded successfully")
    except AssertionError as e:
        print(e)
        print("Ensure your .env file has url, key and secret")
else:
    print('Make sure you have a .env file')

All environment variables loaded successfully


In [2]:
from bbrest import BbRest
bb = BbRest(key, secret, url)

In [3]:
bb.expiration()

'in 44 minutes'

# Use Case : Creating groups for merged classes
Steps involved:
* Getting all course memberships
* Getting child course information
* Generating groups for each section (parent and child)
* Enrolling students into groups for each section

Important note: more logic is needed if you want to check the status of the groups, or if you want to update the groups regularly.  To hear more about that logic, and see an actual tool built around this usecase - I am presenting on this tomorrow.

## Get all course memberships
Note : This is much more difficult with larger classes due to paging.  This is simplified here. BbRest should eventually deal with this, but there's no great way to do this currently.

In [4]:
members = bb.GetCourseMemberships('NAC-DevCon1', params={'limit':'100'}).json()['results']


In [5]:
members[:10]

[{'userId': '_805440_1',
  'courseId': '_332335_1',
  'childCourseId': '_332337_1',
  'dataSourceId': '_2_1',
  'created': '2019-07-22T03:01:36.000Z',
  'availability': {'available': 'Yes'},
  'courseRoleId': 'Student'},
 {'userId': '_805462_1',
  'courseId': '_332335_1',
  'childCourseId': '_332337_1',
  'dataSourceId': '_2_1',
  'created': '2019-07-22T03:01:42.000Z',
  'availability': {'available': 'Yes'},
  'courseRoleId': 'Student'},
 {'userId': '_805447_1',
  'courseId': '_332335_1',
  'dataSourceId': '_2_1',
  'created': '2019-07-22T03:01:38.000Z',
  'availability': {'available': 'Yes'},
  'courseRoleId': 'Student'},
 {'userId': '_805420_1',
  'courseId': '_332335_1',
  'dataSourceId': '_2_1',
  'created': '2019-07-22T03:01:31.000Z',
  'availability': {'available': 'Yes'},
  'courseRoleId': 'Student'},
 {'userId': '_805465_1',
  'courseId': '_332335_1',
  'childCourseId': '_332336_1',
  'dataSourceId': '_2_1',
  'created': '2019-07-22T03:01:43.000Z',
  'availability': {'available

## Get all unique courseIds to create groups

In [6]:
courseIds = set([member.get('childCourseId', member.get('courseId', '')) for member in members])
print(courseIds)

{'_332335_1', '_332336_1', '_332337_1'}


In [7]:
bb.GetCourseChildren('NAC-DevCon1').json()

{'results': [{'id': '_332336_1',
   'dataSourceId': '_2_1',
   'created': '2019-07-22T03:10:50.000Z'},
  {'id': '_332337_1',
   'dataSourceId': '_2_1',
   'created': '2019-07-22T03:10:52.000Z'}]}

## Generate groups for each section (parent and child)

In [8]:
for courseId in courseIds:
    course_info = bb.GetCourse(courseId).json()
    group = {
        'name':course_info['courseId'],
        'externalId':course_info['courseId'],
        'availability':{'available':'Yes'}
    }
    r = bb.CreateGroup(courseId='NAC-DevCon1', payload=group)
    print(r.status_code)

201
201
201


## Enroll students in their sections

In [9]:
students = [member for member in members if member['courseRoleId'] == 'Student']
for student in students:
    userId = student['userId']
    courseId = student.get('childCourseId', student.get('courseId', ''))
    groupId = bb.GetCourse(courseId).json()['courseId']
    r = bb.PutGroupMembership(courseId="NAC-DevCon1", userId=userId, groupId=f'{groupId}')
    print(r.status_code)

201
201
201
201
201
201
201
201
201
201
201
201
201
201
201
201
201
201
201
201
201
201
201
201
201
201
201
201
201
201
201
201
201
201
201
201
201
201
201
201
201
201
201
201
201
201
201
201
201


In [10]:
for course in ['NAC-DevCon1', 'NAC-DevCon2', 'NAC-DevCon3']:
    r = bb.PutGroupMembership(courseId='NAC-DevCon1', userId='m500d520', groupId=course)

In [11]:
bb.GetGroups('NAC-DevCon1').json()

{'results': [{'id': '_138681_1',
   'externalId': 'NAC-DevCon1',
   'parentId': None,
   'name': 'NAC-DevCon1',
   'availability': {'available': 'Yes'},
   'enrollment': {'type': 'InstructorOnly', 'limit': 0}},
  {'id': '_138682_1',
   'externalId': 'NAC-DevCon2',
   'parentId': None,
   'name': 'NAC-DevCon2',
   'availability': {'available': 'Yes'},
   'enrollment': {'type': 'InstructorOnly', 'limit': 0}},
  {'id': '_138683_1',
   'externalId': 'NAC-DevCon3',
   'parentId': None,
   'name': 'NAC-DevCon3',
   'availability': {'available': 'Yes'},
   'enrollment': {'type': 'InstructorOnly', 'limit': 0}}]}

In [12]:
bb.GetGroupMemberships(courseId='NAC-DevCon1',groupId='NAC-DevCon2').json()

{'results': [{'userId': '_805458_1'},
  {'userId': '_805457_1'},
  {'userId': '_805433_1'},
  {'userId': '_805446_1'},
  {'userId': '_805459_1'},
  {'userId': '_805463_1'},
  {'userId': '_805455_1'},
  {'userId': '_702000_1'},
  {'userId': '_805465_1'},
  {'userId': '_805468_1'},
  {'userId': '_805427_1'},
  {'userId': '_805466_1'},
  {'userId': '_805467_1'},
  {'userId': '_805422_1'},
  {'userId': '_805456_1'},
  {'userId': '_805428_1'},
  {'userId': '_805434_1'},
  {'userId': '_805429_1'},
  {'userId': '_805448_1'},
  {'userId': '_805453_1'}]}